### CIS 4360 Computational Methods in Finance

In [4]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from scipy.stats import skew, kurtosis
from math import log, exp, sqrt, pow

**Property of normal distribution**

$$ ln S_T \sim \phi \left[ ln S_0 + \left(\mu - \frac{\sigma^2}{2}\right) T, \sigma^2 T\right]$$

**There is a 95% probability that a normal variable will fall within 1.96 standard deviations**

**The 95% confidence interval of the normal variable will be between (mean - 1.96 stdev) and (mean+1.96 stdev)**  

## Example 15.1

**Consider a stock with an initial price of 40, an expected return of 16\% per annum, and a volatility of 20%. What is the 95% confidence interval of the stock price in 6 months?**

In [2]:
# Example 15.1
S0 = 40
mu = 0.16
sigma = 0.2
T = 6/12

In [3]:
mean = log(S0) + (mu - pow(sigma, 2)/2)*T
stdev = sigma * sqrt(T)
print(mean, stdev)

3.758879454113936 0.14142135623730953


In [4]:
lower_bound = exp(mean - 1.96*stdev)
upper_bound = exp(mean + 1.96*stdev)
print(lower_bound, upper_bound)

32.51474230558975 56.603188205344175


## Example 15.6

**The stock price 6 months from the expiration of an option is 42, exercise price of the option is 40, risk-free rate is 10% and volatility is 20%. What should be the call and put priced at?**

In [16]:
from scipy.stats import norm

In [17]:
S0 = 35
K = 38
r = 0.04
sigma = 0.3
T = 0.5

In [18]:
d1 = (log(S0/K)+(r + sigma**2/2)*T)/(sigma * sqrt(T))
d2 = (log(S0/K)+(r - sigma**2/2)*T)/(sigma * sqrt(T))
print(d1, d2)

-0.18732719156546784 -0.39945922592143207


In [19]:
c = S0* norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
print(c)

delta = 1 * norm.cdf(d1)
print (delta)

2.057457377400917
0.42570205227468616


In [9]:
p = K * exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

In [10]:
p

0.8085993729000922

### Now let's compare that with your Black-Schole Model Implementation

***That is, use your OO Black-Scholes model implementation to calculate the price of the above options. Make sure they reproduce the same answer***

***Import your library***

In [11]:
from blackscholes_model import BlackScholesModel

***Question 1: First create a Stock object with the spot_price and the sigma***

In [12]:

from stock import Stock
import option
import os
import sqlite3
import datetime
opt = option.Option()
opt.data_dir = "./data"
opt.output_dir = os.path.join(opt.data_dir, "daily")
opt.sqlite_db = os.path.join(opt.data_dir, "sqlitedb/Equity.db")


db_file = opt.sqlite_db
db_connection = sqlite3.connect(db_file)
#2023, 10, 1
as_of_date = datetime.date(2023, 10, 1)

stock = Stock(opt, db_connection, sigma=.2, spot_price=42,ticker= 'BABA')



***Question 2: Create instance of the model***

In [1]:
BlackScholes = BlackScholesModel(risk_free_rate=.1,pricing_date=.5)


NameError: name 'BlackScholesModel' is not defined

***Question 3: Now call your model calc_model_price to calculate the above options***

In [14]:

from financial_option import *
EuropeanCallOpt = EuropeanCallOption(stock,T,K)
EuropeanPutOpt = EuropeanPutOption(stock,T,K)

print('call ', BlackScholes.calc_model_price(EuropeanCallOpt))
print('put ', BlackScholes.calc_model_price(EuropeanPutOpt))

call  4.759422392871532
put  0.8085993729000922


#### Testing Call-Put Parity

In [15]:
print('put',BlackScholes.calc_parity_price(EuropeanCallOpt,50))
print('call',BlackScholes.calc_parity_price(EuropeanPutOpt,46.05))

put 46.04917698002856
call 50.000823019971435


***Question 4: Now call your model to calculate ALL the greeks for both options***

In [16]:
# calc delta, gamma, theta, vega for both the call and the put
print("delta + Call")
print(BlackScholes.calc_delta(EuropeanCallOpt))
print("delta + Put")
print(BlackScholes.calc_delta(EuropeanPutOpt))
print('gamma + Call')
print(BlackScholes.calc_gamma(EuropeanCallOpt))
print('gamma + Put')
print(BlackScholes.calc_gamma(EuropeanPutOpt))
print('theta + Call')
print(BlackScholes.calc_theta(EuropeanCallOpt))
print('theta + Put')
print(BlackScholes.calc_theta(EuropeanPutOpt))
print('vega + Call')
print(BlackScholes.calc_vega(EuropeanCallOpt))
print('vega + Put')
print(BlackScholes.calc_vega(EuropeanPutOpt))
print('rho + Call')
print(BlackScholes.calc_rho(EuropeanCallOpt))
print('rho + Put')
print(BlackScholes.calc_rho(EuropeanPutOpt))


delta + Call
0.779131290942669
delta + Put
-0.22086870905733103
gamma + Call
0.04996267040591185
gamma + Put
0.04996267040591185
theta + Call
-4.559092194592627
theta + Put
1.0584711857367115
vega + Call
8.813415059602853
vega + Put
8.813415059602853
rho + Call
13.982045913360283
rho + Put
-5.0425425766539975
